# Create EDD Study Files From Data

This notebook creates the files needed for importing a study into Experiment Data Depot (EDD).

## Inputs and outputs

### Required file to run this notebook:
   - `../data/DBTL7/media_descriptions.csv` - media designs for each of the wells
   
   - `../data/DBTL7/OD.xlsx` - production data from the plate reader


### File generated by running this notebook:
   - `edd_experiment_description.csv`
   
   - `edd_protocol.csv`
 
    
The files are stored in the user defined directory.

## Setup

Importing needed libraries:

In [2]:
import sys
sys.path.append('../media_compiler')

import pandas as pd
import openpyxl

from core import create_media_description

### User parameters

In [3]:
CYCLE = 3

user_params = {
    'media_file': f'../flaviolin yield data/DBTL{CYCLE}/media_descriptions.csv',  
    'measurement_file': f'../flaviolin yield data/DBTL{CYCLE}/OD.xlsx',
    'output_file_path': f'../flaviolin yield data/DBTL{CYCLE}', # Folder for output files,
    'num_replicates': 3,
    'num_designs': 16,
    'protocol_name': ['OD600', 'OD340'],
    'time_point': 48,
    'part_id': 'JBx_193086',
    'media': 'MOPS',
    'culture_volume': 15,
    'well_volume': 1500,
    'shaking_speed': 800,
    'temperature': 30,
    } 


In [4]:
df = pd.read_csv(user_params['media_file'], index_col=0)
df.head()

,MOPS[mM],Tricine[mM],H3BO3[mM],Glucose[mM],K2SO4[mM],K2HPO4[mM],FeSO4[mM],NH4Cl[mM],MgCl2[mM],NaCl[mM],(NH4)6Mo7O24[mM],CoCl2[mM],CuSO4[mM],MnSO4[mM],ZnSO4[mM],Kan[g/l]
Well,,,,,,,,,,,,,,,,
A1,40.0,4.0,0.000400,66.750254,1.920390,6.422140,0.100000,14.059542,0.052000,497.923422,0.000300,0.003000,0.001000,0.0080,0.000010,0.05
B1,40.0,4.0,0.000400,66.750254,1.920390,6.422140,0.100000,14.059542,0.052000,497.923422,0.000300,0.003000,0.001000,0.0080,0.000010,0.05
C1,40.0,4.0,0.000400,66.750254,1.920390,6.422140,0.100000,14.059542,0.052000,497.923422,0.000300,0.003000,0.001000,0.0080,0.000010,0.05
D1,40.0,4.0,0.003116,66.750641,1.895454,6.438428,0.097366,14.030724,0.199599,497.920272,0.000294,0.002793,0.000695,0.0059,0.000027,0.05
E1,40.0,4.0,0.003116,66.750641,1.895454,6.438428,0.097366,14.030724,0.199599,497.920272,0.000294,0.002793,0.000695,0.0059,0.000027,0.05


## Create Line Description

In [5]:
df['Line Description'] = df.apply(create_media_description, axis=1)


In [6]:
df['Line Description']

Well
A1    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
B1    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
C1    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
D1    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
E1    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
F1    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
A2    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
B2    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
C2    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
D2    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
E2    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
F2    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
A3    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
B3    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
C3    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
D3    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
E3    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
F3    MOPS[mM]: 40.000000, Tricine[mM]: 4.0

## Create Line Names

Add metadata for media and replicates to craft Line Names as **C[.]\_W[.]1\_[.]3-R[.]** denoting cycle number, wells occupying the same design and replicate number:

Check if it's row or column order of lines:

In [7]:
column_order = True if df.index[1][0] == 'B' else False

In [8]:
reps = user_params['num_replicates']
num_media_designs = user_params['num_designs']

def linefunction_colum_order(row):
    well = row.name
    if well[0] in 'ABC':
        return f'C{CYCLE}_WA{well[1]}_C{well[1]}-R{row["Replicate"]}'
    else:
        return f'C{CYCLE}_WD{well[1]}_F{well[1]}-R{row["Replicate"]}'                                                                                              

def linefunction_row_order(row):
    well = row.name
    if int(well[1]) < 5:
        return f'C{CYCLE}_W{well[0]}1_{well[0]}4-R{row["Replicate"]}'
    else:
        return f'C{CYCLE}_W{well[0]}5_{well[0]}8-R{row["Replicate"]}'      
    
def linefunction_row_order_triplicates(row):
    well = row.name

    well_names = list(df.index)
    index = well_names.index(well)
    rem = index%3
    name = f'C{CYCLE}_W{well_names[index-rem]}_{well_names[index-rem+2]}-R{row["Replicate"]}'
    print(name)
    return name
    
    

    
df['Replicate'] = [i+1 for _ in range(num_media_designs) for i in range(reps)]
df['Line Name'] = df.apply(linefunction_row_order_triplicates, axis=1)

# if column_order:
#     df['Line Name'] = df.apply(linefunction_colum_order, axis=1)
# else:
#     df['Line Name'] = df.apply(linefunction_row_order, axis=1)

C3_WA1_C1-R1
C3_WA1_C1-R2
C3_WA1_C1-R3
C3_WD1_F1-R1
C3_WD1_F1-R2
C3_WD1_F1-R3
C3_WA2_C2-R1
C3_WA2_C2-R2
C3_WA2_C2-R3
C3_WD2_F2-R1
C3_WD2_F2-R2
C3_WD2_F2-R3
C3_WA3_C3-R1
C3_WA3_C3-R2
C3_WA3_C3-R3
C3_WD3_F3-R1
C3_WD3_F3-R2
C3_WD3_F3-R3
C3_WA4_C4-R1
C3_WA4_C4-R2
C3_WA4_C4-R3
C3_WD4_F4-R1
C3_WD4_F4-R2
C3_WD4_F4-R3
C3_WA5_C5-R1
C3_WA5_C5-R2
C3_WA5_C5-R3
C3_WD5_F5-R1
C3_WD5_F5-R2
C3_WD5_F5-R3
C3_WA6_C6-R1
C3_WA6_C6-R2
C3_WA6_C6-R3
C3_WD6_F6-R1
C3_WD6_F6-R2
C3_WD6_F6-R3
C3_WA7_C7-R1
C3_WA7_C7-R2
C3_WA7_C7-R3
C3_WD7_F7-R1
C3_WD7_F7-R2
C3_WD7_F7-R3
C3_WA8_C8-R1
C3_WA8_C8-R2
C3_WA8_C8-R3
C3_WD8_F8-R1
C3_WD8_F8-R2
C3_WD8_F8-R3


In [66]:
df.head(5)

,MOPS[mM],Tricine[mM],H3BO3[mM],Glucose[mM],K2SO4[mM],K2HPO4[mM],FeSO4[mM],NH4Cl[mM],MgCl2[mM],NaCl[mM],(NH4)6Mo7O24[mM],CoCl2[mM],CuSO4[mM],MnSO4[mM],ZnSO4[mM],Kan[g/l],Line Description,Replicate,Line Name
Well,,,,,,,,,,,,,,,,,,,
A1,40.0,4.0,0.000400,66.750254,1.920390,6.422140,0.100000,14.059542,0.052000,497.923422,0.000300,0.003000,0.001000,0.0080,0.000010,0.05,"MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...",1,C3_WA1_C1-R1
B1,40.0,4.0,0.000400,66.750254,1.920390,6.422140,0.100000,14.059542,0.052000,497.923422,0.000300,0.003000,0.001000,0.0080,0.000010,0.05,"MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...",2,C3_WA1_C1-R2
C1,40.0,4.0,0.000400,66.750254,1.920390,6.422140,0.100000,14.059542,0.052000,497.923422,0.000300,0.003000,0.001000,0.0080,0.000010,0.05,"MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...",3,C3_WA1_C1-R3
D1,40.0,4.0,0.003116,66.750641,1.895454,6.438428,0.097366,14.030724,0.199599,497.920272,0.000294,0.002793,0.000695,0.0059,0.000027,0.05,"MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...",1,C3_WD1_F1-R1
E1,40.0,4.0,0.003116,66.750641,1.895454,6.438428,0.097366,14.030724,0.199599,497.920272,0.000294,0.002793,0.000695,0.0059,0.000027,0.05,"MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...",2,C3_WD1_F1-R2


## Process measurement files

Read measurements file:

In [67]:
df_600 = pd.read_excel(user_params['measurement_file'], sheet_name='600', index_col=0)
df_340 = pd.read_excel(user_params['measurement_file'], sheet_name='340', index_col=0)

# df_600

In [68]:
df_340

,1,2,3,4,5,6,7,8,9,10,11,12
A,0.4317,0.3579,0.5172,0.2099,0.3728,0.2693,0.2892,0.2184,0.0959,0.0916,0.0903,0.0880
B,0.4367,0.3981,0.5063,0.2099,0.3528,0.2789,0.2908,0.2206,0.0916,0.0893,0.0869,0.0838
C,0.4534,0.3976,0.5134,0.2027,0.3570,0.2734,0.2920,0.2115,0.0832,0.0837,0.0806,0.0791
D,0.7424,0.5259,0.4306,0.0001,0.0001,0.2145,0.2759,0.3632,0.0752,0.0767,0.0745,0.0720
E,0.7310,0.5024,0.4185,0.0001,0.0001,0.2065,0.2692,0.3736,0.0710,0.0710,0.0693,0.0662
F,0.7471,0.5054,0.3952,0.0001,0.0001,0.2076,0.2655,0.3572,0.0672,0.0672,0.0651,0.0633
G,0.0609,0.0638,0.0735,0.0688,0.0702,0.0699,0.0698,0.0696,0.0684,0.0665,0.0644,0.0615
H,0.0624,0.0632,0.0655,0.0661,0.0659,0.0649,0.0650,0.0657,0.0659,0.0649,0.0641,0.0623


### Process OD600

Normalize the data to the control well A9 value (water content):

In [69]:
zero_value_600 = df_600.at['A', 9]
df_600.loc[:, df_600.columns] -= zero_value_600
df_600

,1,2,3,4,5,6,7,8,9,10,11,12
A,0.6924,0.4935,1.1831,0.1413,0.0405,0.2515,0.2646,0.1838,0.0000,0.0096,0.0099,0.0090
B,0.6717,0.5710,1.1732,0.1369,0.0354,0.2430,0.2589,0.1848,0.0069,0.0065,0.0066,0.0054
C,0.6893,0.5631,1.1847,0.1336,0.0290,0.2391,0.2528,0.1851,0.0002,0.0019,0.0001,0.0009
D,1.3717,1.1705,0.3602,1.1260,0.9415,0.1698,0.2440,0.0717,-0.0047,-0.0043,-0.0048,-0.0054
E,1.3496,1.1672,0.4091,1.1909,0.9328,0.1867,0.2430,0.4822,-0.0093,-0.0093,-0.0096,-0.0100
F,1.3422,1.1804,0.3566,1.1638,0.9538,0.1719,0.2404,0.1315,-0.0124,-0.0121,-0.0130,-0.0127
G,-0.0130,-0.0131,-0.0121,-0.0121,-0.0125,-0.0130,-0.0125,-0.0126,-0.0126,-0.0132,-0.0133,-0.0139
H,-0.0119,-0.0110,-0.0116,-0.0117,-0.0121,-0.0120,-0.0124,-0.0123,-0.0123,-0.0123,-0.0124,-0.0129


Set negative values to zero, multiply the values by 10 to account for 10x dilution and keep only 8 columns and 6 rows:

In [70]:
df_600[df_600 < 0] = 0
df_600.loc[:, df_600.columns] *= 10
df_600 = df_600.iloc[:6,:8]
df_600

,1,2,3,4,5,6,7,8
A,6.924,4.935,11.831,1.413,0.405,2.515,2.646,1.838
B,6.717,5.710,11.732,1.369,0.354,2.430,2.589,1.848
C,6.893,5.631,11.847,1.336,0.290,2.391,2.528,1.851
D,13.717,11.705,3.602,11.260,9.415,1.698,2.440,0.717
E,13.496,11.672,4.091,11.909,9.328,1.867,2.430,4.822
F,13.422,11.804,3.566,11.638,9.538,1.719,2.404,1.315


### Process OD340

Do the same for OD340, except of multiplication by 10, as the OD340 measurements were taken from non-diluted samples:

In [71]:
zero_value_340 = df_340.at['A', 9]
df_340.loc[:, df_340.columns] -= zero_value_340
df_340[df_340 < 0] = 0
df_340 = df_340.iloc[:6,:8]
df_340

,1,2,3,4,5,6,7,8
A,0.3358,0.2620,0.4213,0.1140,0.2769,0.1734,0.1933,0.1225
B,0.3408,0.3022,0.4104,0.1140,0.2569,0.1830,0.1949,0.1247
C,0.3575,0.3017,0.4175,0.1068,0.2611,0.1775,0.1961,0.1156
D,0.6465,0.4300,0.3347,0.0000,0.0000,0.1186,0.1800,0.2673
E,0.6351,0.4065,0.3226,0.0000,0.0000,0.1106,0.1733,0.2777
F,0.6512,0.4095,0.2993,0.0000,0.0000,0.1117,0.1696,0.2613


Transform wide to long format, matching the order of wells in index to the one from `df`:

In [72]:
if column_order:
    indfcn = lambda x: f'{x.name}{int(x["variable"])}'
    df_600 = df_600.melt(ignore_index=False, value_name='OD600')
    df_340 = df_340.melt(ignore_index=False, value_name='OD340')
else:
    indfcn = lambda x: f'{x["variable"]}{(x.name)}'
    df_600 = df_600.T.melt(ignore_index=False, value_name='OD600')
    df_340 = df_340.T.melt(ignore_index=False, value_name='OD340')

df_600['Well'] = df_600.apply(indfcn, axis=1)
df_340['Well'] = df_340.apply(indfcn, axis=1)

df_600.index = df_600['Well']
df_340.index = df_340['Well']

df_600.drop(columns=['variable', 'Well'], inplace=True)
df_340.drop(columns=['variable', 'Well'], inplace=True)

df_600.head()

,OD600
Well,
A1,6.924
B1,6.717
C1,6.893
D1,13.717
E1,13.496


Add measurements to the main dataframe:

In [82]:
measurOD600 = user_params['protocol_name'][0]
measurOD340 = user_params['protocol_name'][1]
df[measurOD600] = df_600
df[measurOD340] = df_340
df.head(2)


,MOPS[mM],Tricine[mM],H3BO3[mM],Glucose[mM],K2SO4[mM],K2HPO4[mM],FeSO4[mM],NH4Cl[mM],MgCl2[mM],NaCl[mM],...,Media,Part ID,Culture Volume,Flask Volume,Growth Temperature,Shaking speed,Measurement Type,Time,OD600,OD340
Well,,,,,,,,,,,,,,,,,,,,,
A1,40.0,4.0,0.0004,66.750254,1.92039,6.42214,0.1,14.059542,0.052,497.923422,...,MOPS,JBx_193086,15,1500,30,800,OD340,48,6.924,0.3358
B1,40.0,4.0,0.0004,66.750254,1.92039,6.42214,0.1,14.059542,0.052,497.923422,...,MOPS,JBx_193086,15,1500,30,800,OD340,48,6.717,0.3408


## Create EDD Experiment Description File

In [83]:
df['Media'] = user_params['media']
df['Part ID'] = user_params['part_id']
df['Culture Volume'] = user_params['culture_volume']
df['Flask Volume'] = user_params['well_volume']
df['Growth Temperature'] = user_params['temperature']
df['Shaking speed'] = user_params['shaking_speed']
# df['Starting OD'] =
# df['Replicate Count'] = 24


In [84]:
# Invalid columns for now in EDD
# df['Humidity[%]'] = user_params['humidity']
# df['Plate'] = user_params['plate']

In [85]:
exp_descr_file = f'{user_params["output_file_path"]}/edd_experiment_description.xlsx'
df[['Line Name',
    'Line Description',
    'Part ID',
    'Media',
    'Culture Volume',
    'Flask Volume',
    'Growth Temperature',
    'Shaking speed',
]].to_excel(exp_descr_file, index=False)

In [86]:
df.head()

,MOPS[mM],Tricine[mM],H3BO3[mM],Glucose[mM],K2SO4[mM],K2HPO4[mM],FeSO4[mM],NH4Cl[mM],MgCl2[mM],NaCl[mM],...,Media,Part ID,Culture Volume,Flask Volume,Growth Temperature,Shaking speed,Measurement Type,Time,OD600,OD340
Well,,,,,,,,,,,,,,,,,,,,,
A1,40.0,4.0,0.000400,66.750254,1.920390,6.422140,0.100000,14.059542,0.052000,497.923422,...,MOPS,JBx_193086,15,1500,30,800,OD340,48,6.924,0.3358
B1,40.0,4.0,0.000400,66.750254,1.920390,6.422140,0.100000,14.059542,0.052000,497.923422,...,MOPS,JBx_193086,15,1500,30,800,OD340,48,6.717,0.3408
C1,40.0,4.0,0.000400,66.750254,1.920390,6.422140,0.100000,14.059542,0.052000,497.923422,...,MOPS,JBx_193086,15,1500,30,800,OD340,48,6.893,0.3575
D1,40.0,4.0,0.003116,66.750641,1.895454,6.438428,0.097366,14.030724,0.199599,497.920272,...,MOPS,JBx_193086,15,1500,30,800,OD340,48,13.717,0.6465
E1,40.0,4.0,0.003116,66.750641,1.895454,6.438428,0.097366,14.030724,0.199599,497.920272,...,MOPS,JBx_193086,15,1500,30,800,OD340,48,13.496,0.6351


## Create EDD Measurement File

OD600

In [87]:
measurement_file = f'{user_params["output_file_path"]}/edd_{measurOD600}.xlsx'
df['Measurement Type'] = measurOD600


df['Time'] = user_params['time_point']
df['Value'] = df[measurOD600]
df['Units'] = 'n/a'
df[['Line Name', 'Measurement Type', 'Time', 'Value', 'Units']].to_excel(measurement_file, index=False)

OD340

In [88]:
# measurement_file = f'{user_params["output_file_path"]}/edd_{measurOD340}.xlsx'

# df['Time'] = user_params['time_point']
# df['Value'] = df[measurOD340]
# df['Units'] = 'n/a'
# df[['Line Name', 'Measurement Type', 'Time', 'Value', 'Units']].to_excel(measurement_file, index=False)

OD

In [89]:
measurement_file = f'{user_params["output_file_path"]}/edd_OD.xlsx'
df['Measurement Type'] = measurOD340

df['Time'] = user_params['time_point']
df['Value'] = df[measurOD340]
df['Units'] = 'n/a'
df[['Line Name', 'Measurement Type', 'Time', 'Value', 'Units']].to_excel(measurement_file, index=False)